In [ ]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Load saved models
next_value_model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Project-1/Models/next_value_prediction_model-4.keras")
type_classification_model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Project-1/Models/sequence_classification_model.keras")
coefficient_model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Project-1/Models/coefficient_prediction_model-2.keras")

In [ ]:
# Load preprocessed scalers and encoders
with open("/content/drive/MyDrive/Colab Notebooks/Project-1/Scalers/preprocessed_data.pkl", "rb") as f:
    preprocessed_data = pickle.load(f)

seq_scaler = preprocessed_data["seq_scaler"]
next_scaler = preprocessed_data["next_scaler"]
coeff_scaler = preprocessed_data["coeff_scaler"]
label_encoder = preprocessed_data["label_encoder"]

In [ ]:
# Define function to preprocess input sequence
def preprocess_sequence(sequence, max_length=20):
    sequence = np.array(sequence).reshape(1, -1)  # Convert to NumPy array and reshape
    sequence_padded = pad_sequences(sequence, maxlen=max_length, padding="post")  # Pad sequence
    sequence_scaled = seq_scaler.transform(sequence_padded)  # Apply scaling
    sequence_scaled = np.expand_dims(sequence_scaled, axis=-1)  # Reshape for model
    return sequence_scaled

In [ ]:
# Define function for making predictions
def predict_all(sequence):
    # Preprocess input sequence
    processed_sequence = preprocess_sequence(sequence)

    # Predict next value
    next_value_scaled = next_value_model.predict(processed_sequence)
    try:
        next_value_transformed = next_scaler.inverse_transform(next_value_scaled)
    except ValueError as e:
        print(f"Error in inverse transformation: {e}")
        next_value_transformed = next_value_scaled

    next_value_transformed = np.clip(next_value_transformed, a_min=0, a_max=None)
    next_value_original = np.expm1(next_value_transformed)

    next_value_final = np.round(next_value_original)

    # Predict sequence type
    type_probs = type_classification_model.predict(processed_sequence)
    predicted_type = label_encoder.inverse_transform([np.argmax(type_probs)])[0]

    # Predict coefficients
    coeffs_scaled = coefficient_model.predict(processed_sequence)
    coeffs = np.round(coeff_scaler.inverse_transform(coeffs_scaled)[0])

    # Display results
    print("\n=== Prediction Results ===")
    print(f"Input Sequence: {sequence}")
    print(f"Predicted Next Value: {next_value_final}")
    print(f"Predicted Sequence Type: {predicted_type}")
    print(f"Predicted Coefficients: {coeffs}")

In [ ]:
# Function to get user input
def get_user_input():
    while True:
        try:
            user_input = input("\nEnter a sequence of numbers separated by commas (or type 'exit' to quit): ")
            if user_input.lower() == "exit":
                print("Exiting the program. Have a great day! 🚀")
                break
            sequence = list(map(int, user_input.split(",")))
            if len(sequence) < 3:
                print("Please enter at least 3 numbers for a valid sequence!")
                continue
            predict_all(sequence)
        except ValueError:
            print("Invalid input! Please enter numbers separated by commas.")

In [ ]:
# Start the user input loop
get_user_input()


Enter a sequence of numbers separated by commas (or type 'exit' to quit): 12,19,24,39,52
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step

=== Prediction Results ===
Input Sequence: [12, 19, 24, 39, 52]
Predicted Next Value: [[62.]]
Predicted Sequence Type: Arithmetic
Predicted Coefficients: [11.  8.  0.  0.]

Enter a sequence of numbers separated by commas (or type 'exit' to quit): 8,36,104,230
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step

=== Prediction Results ===
Input Sequence: [8, 36, 104, 230]
Predicted Next Value: [[434.]]
Predicted Sequence Type: Cubic
Predicted Coefficients: [2. 2. 2. 1.]

Enter a sequence of numbers separated by commas (or type 'exit' to quit): 8,18,32,50,72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step

=== Prediction Results ===
Input Sequence: [8, 18, 32, 